In [3]:
#ETtoday

import requests
import json
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd
from lxml import etree
import json
import time,random,datetime,re
from tomorrow import threads
from datetime import datetime
from pymongo import MongoClient
import pymongo

#搜尋網址基本連結
ua = UserAgent()
headers = {"User-Agent":ua.random}

def GetAllLinks():

    url = "https://www.ettoday.net/news/focus/政治/"
    res = requests.get(url,headers=headers)
    soup = BeautifulSoup(res.text)

    links = []
    #first
    for i in soup.find('div',class_="block block_1 infinite_scroll").find_all("a",class_="pic"):
    #     print("https://www.ettoday.net/news"+i.get('href'))
        links.append("https://www.ettoday.net"+i.get('href'))

    page = 1
    while True:
        data = {

            'offset': page,
            'tPage': 2,
            'tFile': "1.json",
            'tOt': 0,
            'tSi': 24,
            'tAr': 0

        }

        url = "https://www.ettoday.net/show_roll.php"
        res = requests.post(url,headers=headers,data=data)
        soup = BeautifulSoup(res.text)

        if len(soup) == 0:
            break
        for i in soup.find_all("a",class_="pic"):
            links.append("https://www.ettoday.net"+i.get("href"))
        page += 1
    return links

In [4]:
#爬內文

ua = UserAgent()
headers = {"User-Agent":ua.random}


def ETtodayCrawler(response):

    ua = UserAgent()
    headers = {"User-Agent":ua.random}
    soup = BeautifulSoup(response.text)
    ndf = pd.DataFrame(data = [{'Source':'News',
                                'Site':'ETtoday',
                                'Url':soup.find_all('link')[0]['href'],
                                'CreateTime':datetime.strptime(soup.find('time',{'class':'date'}).text.replace(" ",""),'\n%Y年%m月%d日%H:%M'),
                                'Author':'無作者',
                                'Title':soup.find('h1', attrs={'class':'title'}).text,
                                'Content':"".join(i.text for i in soup.find('div',attrs={'class':'story'}).find_all('p')).replace("\n",""),
                                'Tag':soup.find('meta',{'name':'news_keywords'})['content'],
                                }],
                       columns = ['Source','Site','Url','CreateTime','Author','Title','Content','Tag']) 
    return ndf

In [5]:
#測試抓取一篇文章並存為是否成功

# url = "https://www.ettoday.net/news/20191125/1587285.htm"
# res = requests.get(url,headers=headers)
# ETtodayCrawler(res)

In [6]:
# 多線程爬蟲定義
@threads(100)
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

In [7]:
def CrawlAllNews(links):
    # 爬取全部連結的新聞內容
    print('There are {} links in pages.'.format(len(links)))
    
    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    allData = []
    for response in responses:
        try:
            ndf = ETtodayCrawler(response)
            allData.append(ndf)
        except:
            pass
    df = pd.concat(allData, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df 


In [8]:
def dataframe_to_mongo_test(select_df):
    user_name = "test"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "test"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.test   #DB_name
    collection = db.test_news  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")


In [9]:
def dataframe_to_mongo(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "192.168.1.126"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")
        

In [10]:
def dataframe_to_mongo_political(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")
        

In [11]:
links = GetAllLinks() 
df = CrawlAllNews(links)
dataframe_to_mongo_political(df)

There are 96 links in pages.
There are 94 News in DataFrame.
